# Capital Gain Calculator (FIFO Method)
Adapted from VBA to Python

### Structures
1. Transaction queue
    - HashMap (key is Asset, dict is tuple of two deques)
2. Transactions
    - 6 properties: Timestamp, Asset (eg. BTC), Type (buy or sell), Units, Total Amount ($), IRS ID (eg. Gemi 1)

### Outline
1. Validate transaction log CSV
    - Sort CSV by Timestamp property
    - Ensure valid Type property and corresponding Units property sign
    - Ensure other properties properties
2. Input data from transaction log CSV file into a buy transaction queue and sell transaction queue for each asset (eg. transaction log with BTC and ETH transactions -> BTC-Buy, BTC-Sell, ETH-Buy, Eth-Sell)
3. For each asset, run FIFO algorithm
    - While there are still transactions in the sell transaction queue, match front of sell queue with front of buy queue transaction (verifying buy transaction Timestamp property is before sell transaction)
    - When a match is found, add write buy-sell transaction to output CSV file
    - Update remaining balance (Units and Total Amount property) on buy/sell transaction and/or remove empty transaction(s) from respective queue
    - Run until the sell transaction queue is empty, the remaining buy transaction(s) are the carryover
4. Create summary report


Question:

FIFO method only matters in the case that there is carryover. If there is no carryover, that means every buy has a sell?? Could we just say in the case of carryover, look back at where the most recent buy transactions were and conclude that those are the carryover inventory still in possession. 

Actually I'm not entirely sure, it may be that the timing of certain buy/sell affects capital gain.

# 1. Read and Validate Transaction Log CSV

In [395]:
import csv
from datetime import datetime

# Open input CSV file
file_name = "small_example.csv"
file = open(file_name, 'r')
transaction_log = csv.DictReader(file)

# Sort transaction log by Timestamp property (must have format mm/dd/yyyy hh:mm:ss)
transaction_log = sorted(transaction_log, key=lambda d: datetime.strptime(d['Timestamp'], "%m/%d/%Y %H:%M:%S"))

# Change datatypes of Units and Total Amount to float
for tx in transaction_log:
    tx['Units'] = float(tx['Units'])
    tx['Total Amount'] = float(tx['Total Amount'])

# 2. Input Data into Transaction Queues

In [396]:
from collections import deque

# Create Transaction Queues
asset_map = {}

# Define buy and sell constants
BUY = 0
SELL = 1

for tx in transaction_log:
    # Create buy and sell deques for each asset
    asset = tx['Asset']
    if asset not in asset_map:
        buy_deque = deque()
        sell_deque = deque()
        asset_map[asset] = (buy_deque, sell_deque)
    
    # Add transaction to respective deque
    if tx['Type'] == "Buy":
        asset_map[asset][BUY].appendleft(tx)
    else:
        asset_map[asset][SELL].appendleft(tx)
        
# Close input CSV File
file.close()

# 3. Run FIFO Transaction Matching

In [397]:
# Open output CSV file
file_name = "fifo.csv"
file = open(file_name, 'w', encoding='UTF8', newline='')
writer = csv.writer(file)

# Define fieldnames
fieldnames = ['Asset', 'Date Purchased', 'Date Sold', 'Units', 'Sale Price', 'Basis', 'Gain / Loss', 'IRS ID Buy', 'IRS ID Sell']
writer = csv.DictWriter(file, fieldnames=fieldnames)

# Write fieldnames into output CSV file
writer.writeheader()

# Create dictionary to store short and long term capital gain/loss and volume information
year_summary = {}

# Loop through each asset
for asset in asset_map:
    print(asset)
    print("--------------------------------------------")
    capital_gain, buy_volume, sell_volume = 0, 0, 0
    
    # Create dictionary to store match transaction information
    match = {}
    match['Asset'] = asset
    
    # Run until there are no more transactions in sell deque
    while asset_map[asset][SELL]:
        
        # Get earliest non-matched sell transaction
        sell_tx = asset_map[asset][SELL].pop()
        
        # rounding down to 0????
        if abs(sell_tx['Units']) < .0000001:
            print(sell_tx['Units'])
            continue
        
        # Catch error here: While sell deque is not empty, buy deque must not be empty
        assert(asset_map[asset][BUY])
        
        # Get earliest non-matched buy transaction
        buy_tx = asset_map[asset][BUY].pop()
        
        # Catch error here: Buy_tx['Timestamp'] must be before sell_tx['Timestamp']
        assert (datetime.strptime(buy_tx['Timestamp'], "%m/%d/%Y %H:%M:%S") <= datetime.strptime(sell_tx['Timestamp'], "%m/%d/%Y %H:%M:%S"))
        
        # Populate match dictionary
        match['IRS ID Buy'] = buy_tx['IRS ID']
        match['IRS ID Sell'] = sell_tx['IRS ID']
        match['Date Purchased'] = buy_tx['Timestamp']
        match['Date Sold'] = sell_tx['Timestamp']
        
        # Sell transaction units are greater, so empty buy transaction
        if abs(sell_tx['Units']) > buy_tx['Units']:
            
            # Calculate pro rata sale price
            pro_rata_sale_price = buy_tx['Units'] / abs(sell_tx['Units']) * sell_tx['Total Amount']
            
            # Populate match dictionary
            match['Units'] = buy_tx['Units']
            match['Sale Price'] = pro_rata_sale_price
            match['Basis'] = buy_tx['Total Amount']
            
            # Update sell transaction information and put back into deque
            sell_tx['Units'] = sell_tx['Units'] + buy_tx['Units']
            sell_tx['Total Amount'] = sell_tx['Total Amount'] - pro_rata_sale_price
            
            asset_map[asset][SELL].append(sell_tx)
        
        # Buy transaction units are greater, so empty sell transaction
        elif abs(sell_tx['Units']) < buy_tx['Units']:
            
            # Calculate pro rata basis
            pro_rata_basis = abs(sell_tx['Units']) / buy_tx['Units'] * buy_tx['Total Amount']
            
            # Populate match dictionary
            match['Units'] = abs(sell_tx['Units'])
            match['Sale Price'] = sell_tx['Total Amount']
            match['Basis'] = pro_rata_basis
            
            # Update buy transaction information and put back into deque
            buy_tx['Units'] = buy_tx['Units'] + sell_tx['Units']
            buy_tx['Total Amount'] = buy_tx['Total Amount'] - pro_rata_basis
            asset_map[asset][BUY].append(buy_tx)
        
        # Transaction units are the same
        else:
            
            # Populate match dictionary
            match['Units'] = buy_tx['Units']
            match['Sale Price'] = sell_tx['Total Amount']
            match['Basis'] = buy_tx['Total Amount']
            
        # Calculate gain or loss
        match['Gain / Loss'] = match['Sale Price'] - match['Basis']

        # Write transaction match into output CSV file
        writer.writerow(match)
        
        
        
        

        
    # Carryover is remaining transactions in buy deque
    while asset_map[asset][BUY]:
        buy_tx = asset_map[asset][BUY].pop()
        
        # rounding down to 0????
        if abs(buy_tx['Units']) < .0000001:
            print(buy_tx['Units'])
            continue
        
        match['IRS ID Buy'] = buy_tx['IRS ID']
        match['IRS ID Sell'] = '-'
        match['Date Purchased'] = buy_tx['Timestamp']
        match['Date Sold'] = '-'
        match['Units'] = buy_tx['Units']
        match['Sale Price'] = '-'
        match['Basis'] = buy_tx['Total Amount']
        match['Gain / Loss'] = '-'
        
        # Write carryover into output CSV file
        writer.writerow(match)
        
        buy_volume += buy_tx['Total Amount']
        
    
print("Capital Gain:", capital_gain)
print("Buy Volume:", buy_volume)
print("Sell Volume:", sell_volume)
print("\n")
    
    
# Close output CSV File
file.close()

BTC
--------------------------------------------
Capital Gain: 0.0
Buy Volume: 8.0
Sell Volume: 8.0


ETH
--------------------------------------------
Capital Gain: 0.0
Buy Volume: 3.0
Sell Volume: 2.0




# 4. Create Summary Report